In [1]:
%pip install openai
from openai import OpenAI
import os
from utils import *

#setup openai client
client = OpenAI(
    api_key = os.environ['OPENAI_API_KEY']
)

Note: you may need to restart the kernel to use updated packages.


In [4]:
def adversarial_edit(article_text, client):
    """
    Takes in a Wikipedia article text and generates an adversarial edit using OpenAI's GPT-4 model.
    
    Parameters:
    - article_text (str): The text of the Wikipedia article to be edited.
    - client (openai.ApiClient): An instance of the OpenAI API client.
    
    Returns:
    - str: The adversarially edited article text.
    """
    prompt = (
        "Here is a wikipedia article: \"" + article_text + "\". "
        "I want to adversarially edit this article so that the information is no longer correct.\n"
        "For example, if my article is \"George Washington was born in 1732\", then after editing, "
        "the new incorrect article might be \"George Washington was born in 1700\". \n"
        "Please make a maximum of 5-7 changes to the article to make it incorrect.\n"
        "Please return the edited adversarial article by itself as the singular output, with no text surrounding it."
    )
    
    while True:
        try:
            completion = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {
                        "role": "system",
                        "content": "You are an AI. Please answer the following questions as if you were an expert on the subject.",
                    },
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                temperature = 0.5,
                max_tokens = 1000,
                n=1,
            )
            break
        except Exception as e:
            print("Error:", e)
    
    # Extract and return the edited article content
    return completion.choices[0].message.content

In [5]:
# load datasets
post_2024_articles = load_json('../datasets/post_2024_articles.json')
pre_2021_articles = load_json('../datasets/pre_2021_articles.json')

# load checkpoint
if not os.path.exists('../datasets/adversarial_articles_post_2024.json'):
    adversarial_articles_post_2024 = {}
else:
    adversarial_articles_post_2024 = load_json('../datasets/adversarial_articles_post_2024.json')

if not os.path.exists('../datasets/adversarial_articles_pre_2021.json'):
    adversarial_articles_pre_2021 = {}
else:
    adversarial_articles_pre_2021 = load_json('../datasets/adversarial_articles_pre_2021.json')


In [6]:
# Create adverarial articles
for title, article in post_2024_articles.items():
    if title not in adversarial_articles_post_2024:
        adversarial_articles_post_2024[title] = adversarial_edit(article, client)
        print(len(adversarial_articles_post_2024))

for title, article in pre_2021_articles.items():
    if title not in adversarial_articles_pre_2021:
        adversarial_articles_pre_2021[title] = adversarial_edit(article, client)
        print(len(adversarial_articles_pre_2021))

In [10]:
print(len(adversarial_articles_post_2024))
print(len(adversarial_articles_pre_2021))

5000
5000


In [11]:
with open('../datasets/adversarial_articles_post_2024.json', 'w', encoding = 'utf-8') as f:
    json.dump(adversarial_articles_post_2024, f, ensure_ascii=False, indent=4)
with open('../datasets/adversarial_articles_pre_2021.json', 'w', encoding = 'utf-8') as f:
    json.dump(adversarial_articles_pre_2021, f, ensure_ascii=False, indent=4)